In [1]:
import pandas as pd
import numpy as np
import warnings
import random, math, os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../utils/")
from recsys_metric import mrr
warnings.filterwarnings('ignore')

In [2]:
def get_data(train_file_path, val_file_path, overview=False, concat_val=False):
    # 读取数据
    
    trn_raw_data = pd.read_csv(train_file_path, sep=',', engine='python')
    val_raw_data = pd.read_csv(val_file_path, sep=',', engine='python')
    if concat_val:
        trn_raw_data = pd.concat([trn_raw_data, val_raw_data])
    
    # 分割训练和验证集
    
    trn_data = trn_raw_data.groupby('session_id')['item_id'].apply(list).reset_index()
    val_data = val_raw_data.groupby('session_id')['item_id'].apply(list).reset_index()

    trn_user_items = {}
    val_user_items = {}
    
    # 将数组构造成字典的形式{session_id: [item_id1, item_id2,...,item_idn]}
    for session, item in zip(*(list(trn_data['session_id']), list(trn_data['item_id']))):
        trn_user_items[session] = set(item)

    for session, item in zip(*(list(val_data['session_id']), list(val_data['item_id']))):
        val_user_items[session] = set(item)
    
    if overview:
        print(f'trn_raw_data:\n {trn_raw_data} \n val_raw_data:\n {val_raw_data}')
        print('trn_data:\n',trn_data)
        print('val_data:\n', val_data)
    
    return trn_user_items, val_user_items


In [3]:
def Item_CF(trn_user_items, val_user_items, K, N, candidate_items=None, overview=False, prediction=False):
    '''
    trn_user_items: 表示训练数据，格式为：{user_id1: [item_id1, item_id2,...,item_idn], user_id2...}
    val_user_items: 表示验证数据，格式为：{user_id1: [item_id1, item_id2,...,item_idn], user_id2...}
    K: Ｋ表示的是相似商品的数量，为每个用户交互的每个商品都选择其最相似的K个商品
    N: N表示的是给用户推荐的商品数量，给每个用户推荐相似度最大的N个商品
    '''

    # 建立user->item的倒排表
    # 倒排表的格式为: {user_id1: [item_id1, item_id2,...,item_idn], user_id2: ...} 也就是每个用户交互过的所有商品集合
    # 由于输入的训练数据trn_user_items,本身就是这中格式的，所以这里不需要进行额外的计算
    

    # 计算商品协同过滤矩阵
    # 即利用user-items倒排表统计商品与商品之间被共同的用户交互的次数
    # 商品协同过滤矩阵的表示形式为：sim = {item_id1: {item_id２: num1, item_id3: num3,...}, item_id３: {item_id４: num２,...}, ...}
    # 商品协同过滤矩阵是一个双层的字典，用来表示商品之间共同交互的用户数量
    # 在计算商品协同过滤矩阵的同时还需要记录每个商品被多少不同用户交互的次数，其表示形式为: num = {item_id1：num1, item_id２:num2, ...}
    sim = {}#两个物品同时被某些用户喜欢的总次数(不止一个用户)
    num = {}#每个物品的总共的 被喜欢(交互)数
    print('构建相似性矩阵．．．')
    #遍历每一个user交互过的items列表，统计两个不同的item共同被user(不止一个，而是trn_user_items中的所有user)交互的总次数
    for uid, items in tqdm(trn_user_items.items()):
        #对于当前items列表中的每一个item
        for i in items:
            #统计item i 被用户交互的总次数
            if i not in num:
                num[i] = 0
            num[i] += 1
            #统计 item i 和 item j 被共同交互的总次数
            if i not in sim:
                sim[i] = {}
            for j in items:
                if j not in sim[i]:
                    sim[i][j] = 0
                if i != j:
                    sim[i][j] += 1/math.log1p(len(items) * 1.)
    
    # 计算物品的相似度矩阵
    # 商品协同过滤矩阵其实相当于是余弦相似度的分子部分,还需要除以分母,即两个商品被交互的用户数量的乘积
    # 两个商品被交互的用户数量就是上面统计的num字典
    print('计算协同过滤矩阵．．．')
    for i, items in tqdm(sim.items()):
        for j, score in items.items():
            if i != j:
                sim[i][j] = score / math.sqrt(num[i] * num[j])
                
        
    # 对验证数据中的每个用户进行TopN推荐
    # 在对用户进行推荐之前需要先通过商品相似度矩阵得到 与 当前测试用户交互过的商品最相似的前K个商品，
    # 然后对这K个用户交互的商品中除当前测试用户训练集中交互过的商品以外的商品计算最终的相似度分数
    # 最终推荐的候选商品的相似度分数是由多个相似商品对该商品分数的一个累加和
    items_rank = {}
    print('给用户进行推荐．．．')
    for uid, _ in tqdm(val_user_items.items()):
        items_rank[uid] = {} # 存储用户候选的推荐商品
        for hist_item in trn_user_items[uid]: # 遍历该用户历史喜欢的商品，用来下面寻找其相似的商品
            # 回顾：sim = {item_id1: {item_id２: num1, item_id3: num3,...}, item_id３: {item_id４: num２,...}, ...}
            # 回顾：trn_user_items = {user_id1: [item_id1, item_id2,...,item_idn], user_id2: ...} 
            #print(sim[hist_item])# {item_id２: num1, item_id3: num3,...}
            for item, score in sorted(sim[hist_item].items(), key=lambda x: x[1], reverse=True)[:K]:
                if item not in trn_user_items[uid]: # 进行推荐的商品一定不能在历史喜欢商品中出现
                    #计算当前uid对当前item的打分
                    if item not in items_rank[uid]:
                        items_rank[uid][item] = 0
                    items_rank[uid][item] += score
    
    print('为每个用户筛选出相似度分数最高的Ｎ个商品...')
    if not prediction:
        #print(items_rank)#{user1:{item1:score1,item2:score2,...},user2:{item6:score6,item9:score9,...}}
        items_rank = {k: sorted(v.items(), key=lambda x: x[1], reverse=True)[:N] for k, v in items_rank.items()}
        items_rank = {k: set([x[0] for x in v]) for k, v in items_rank.items()}
    else:
        items_rank = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in items_rank.items()}
        items_rank = {k: set([x[0] for x in v if x[0] in candidate_items][:N]) for k, v in items_rank.items()}
        
        
        
    
    if overview:
        #查看一下sim   
        for k ,v in sim.items():
            print(f"similarity[{k}]:") 
            print(v)
            break
            
    return items_rank, sim



In [4]:
def rank_itemcf(target_dict, overview=False):
    item_id_rank_arr = pd.DataFrame.from_dict(target_dict, orient='index').reset_index().rename(columns={'index': 'session_id'}).fillna(-1).astype(int)
    lack_col_num = 102 - len(item_id_rank_arr.columns)
    for i in range(100-lack_col_num,100):
        item_id_rank_arr[i]=-1
     
    if overview:
        print(f"lack_col_num:{lack_col_num}")
        display(item_id_rank_arr)
    
    res_df = pd.melt(item_id_rank_arr, id_vars=['session_id'], value_vars=range(100)).sort_values(['session_id','variable']).rename(columns={'value': 'item_id'}).drop(['variable'], axis=1)
    res_df["rank"] = (list(range(1,101))*len(target_dict))
    
    
    return res_df

# evaluation

In [5]:
# 读取数据
train_file_path = '../../recsys2022/train_sessions.csv'
val_file_path = '../datasets/train_last_1m.csv'
trn_user_items, val_user_items = get_data(train_file_path, val_file_path, concat_val=False)
print(f'trn_user_items len:{len(trn_user_items)},val_user_items len:{len(val_user_items)}')


# 計算ItemCF
k=3
n=100
rec_items, sim = Item_CF(trn_user_items, val_user_items, k, n, prediction=False)


# rank item & output rank result
eva_res = rank_itemcf(rec_items)

trn_user_items len:1000000,val_user_items len:47320
构建相似性矩阵．．．


100%|██████████████████████████████| 1000000/1000000 [00:21<00:00, 45997.88it/s]


计算协同过滤矩阵．．．


100%|███████████████████████████████████| 23496/23496 [00:06<00:00, 3895.38it/s]


给用户进行推荐．．．


100%|████████████████████████████████████| 47320/47320 [01:45<00:00, 450.27it/s]


为每个用户筛选出相似度分数最高的Ｎ个商品...


In [6]:
len(sim)

23496

In [7]:
np.zeros((1,len(sim)),dtype=int)

array([[0, 0, 0, ..., 0, 0, 0]])

In [11]:
ddic = {(key):np.zeros((len(sim)),dtype=int) for key in sim.keys()}

In [12]:
df = pd.DataFrame(ddic)
df["idx"] = list(df.columns)
df = df.set_index("idx")
df.head()

,9655,15654,4026,2507,18316,20033,4385,27937,12804,6341,...,20114,28134,1015,14095,3416,15964,27863,7735,21419,idx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9655
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15654
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4026
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2507
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18316


In [23]:
test_key = list(sim.keys())[:5]
for elem in test_key:
    for sims in sim[elem]:
        df[elem].apply(lambda x:print(x.index) )

AttributeError: 'int' object has no attribute 'index'

In [48]:
eva_label = pd.read_csv('../datasets/purchases_last_1m.csv')

mrr_score = mrr(eva_res, eva_label, overview=False)

print(mrr_score)
display(f"eva_res shape:{eva_res.shape}", eva_res.iloc[95:105,:])

0.12267361692640234


'eva_res shape:(4732000, 3)'

,session_id,item_id,rank
4495400,115,-1,96
4542720,115,-1,97
4590040,115,-1,98
4637360,115,-1,99
4684680,115,-1,100
1,261,28128,1
47321,261,16130,2
94641,261,10534,3
141961,261,11528,4
189281,261,16404,5


# Output

In [ ]:
# 读取数据
train_file_path = '../../recsys2022/train_sessions_purchases.csv'
val_file_path = '../../recsys2022/test_leaderboard_sessions.csv'
trn_user_items, val_user_items = get_data(train_file_path, val_file_path, concat_val=True)
print(f'trn_user_items len:{len(trn_user_items)},val_user_items len:{len(val_user_items)}')


# 計算ItemCF
candidate_items = pd.read_csv('../../recsys2022/candidate_items.csv')['item_id'].tolist()
k=5 # 4 is also acceptable  
n=100
rec_items, sim = Item_CF(trn_user_items, val_user_items, k, n, candidate_items=candidate_items, prediction=True)


# rank item & output rank result
res = rank_itemcf(rec_items)
res.to_csv("../datasets/results/leader_itemcf_k5_train_purchases_test_candidate_filter_0608.csv", index=False)

In [26]:
# view_session_id = 113
view_session_id = 115
print(rec_items[view_session_id])
print(val_user_items[view_session_id])
print(trn_user_items[view_session_id])
print(sim[list(trn_user_items[view_session_id])[0]])

set()
{19705, 10442, 15969, 16631}
{19705, 10442, 15969, 16631}
{19705: 0, 10442: 0.13829006285626602, 15969: 0.22672126273502682, 16631: 0.10119471737236646, 5798: 0.0012472320256899196, 264: 0.004184726485756, 24872: 0.002895678209357151, 16142: 0.0026340891992597437, 10831: 0.0012536117707700877, 3955: 0.0012552220627258385, 8406: 0.001969434556370532, 21756: 0.003334235708243486, 11998: 0.001863086597070546, 12141: 0.0015662441819701458, 2353: 0.001632991776429401, 22747: 0.0012813141555206718, 5375: 0.0027550213805843985, 16544: 0.0034789659045666946, 25103: 0.00481668739092348, 26896: 0.0055618315235038945, 21330: 0.006869944754049413, 21939: 0.0032020639754251922, 23412: 0.005968989772677565, 4377: 0.006026092494087058, 3440: 0.008835377726720502, 6491: 0.002026832350081482, 8781: 0.05365644456646822, 8707: 0.0017089001771144194, 11693: 0.0027320066317323565, 18290: 0.0021133863987739185, 13243: 0.0020567546201212586, 20156: 0.0024965904917487935, 23648: 0.01236934009156488, 803

In [ ]:
rec_items